# Mobile Price Prediction

### importing libraries

In [ ]:
import sagemaker
import pandas as pd
import boto3   # boto3 library is used to connect s3 bucket
from sklearn.model_selection import train_test_split

### setting up the system

In [ ]:
sm_boto3 = boto3.client('sagemaker')  
# boto3 client is used to connect sagemaker
sess = sagemaker.Session()
# sagemaker session is used to create sagemaker session
bucket = 'XXXXXXXXXXXX'
# bucket name is given
print("Using bucket " + bucket)

### loading data and basic preprocessing

In [ ]:
data = pd.read_csv("Data/train.csv")

In [ ]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 25)

In [ ]:
data.head()

Mobile Phones Category - 
   * 0(low cost), 
   * 1(medium cost), 
   * 2(high cost) and 
   * 3(very high cost)

In [ ]:
data.info()

In [ ]:
data.price_range.value_counts(normalize=True)

In [ ]:
features = list(data.columns)
features

In [ ]:
label = features.pop(-1)
label

In [ ]:
X = data[features]
y = data[label]

In [ ]:
X.shape, y.shape

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
y.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, stratify=y, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
## creating dataframes

trainX = pd.DataFrame(X_train)
trainX['label'] = y_train

testX = pd.DataFrame(X_test)
testX['label'] = y_test

In [ ]:
trainX.shape, testX.shape

In [ ]:
trainX.head()

## Data Ingestion

In [ ]:
## saving it to csv file

trainX.to_csv("Data/train-V-1.csv", index = False)
testX.to_csv("Data/test-V-1.csv", index = False)

In [ ]:
bucket

In [ ]:
## send data to S3, SageMaker will take training data from S3

sk_prefix = 'sagemaker/mobile_price_classification/sklearncontainer'

trainpath = sess.upload_data(
    path = "Data/train-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

testpath = sess.upload_data(
    path='Data/test-V-1.csv', bucket=bucket, key_prefix=sk_prefix
)

In [ ]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO 
import argparse
import joblib
import os
import numpy as np
import pandas as pd
    
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf
    
if __name__ == "__main__":

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--n_jobs", type=int, default=-1)
    parser.add_argument("--verbose", type=int, default=3)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
  
    print("Training RandomForest Model.....")
    print()
    model =  RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose = args.verbose ,n_jobs=args.n_jobs)
    model.fit(X_train, y_train)
    print()
    

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)


In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# RandomForestClassifier().get_params()

In [ ]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    # role=sagemaker.get_execution_role(),
    role = "xxxxxxxxxxxx",
    instance_count = 1,
    instance_type = "ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name = "sklearn-mobile-price-classification",
    hyperparameters = {
        "n_estimators":100,
        "n_jobs":-1,
        "verbose":3,
        "random_state":0
    },
    use_spot_instances=True,
    max_wait=7200,
    max_run = 3600
)

In [ ]:
## launch training job, with asynchronous call
sklearn_estimator.fit({"train":trainpath, "test":testpath}, wait = True)

In [ ]:
sklearn_estimator.latest_training_job.wait(logs='None')

artifact = sm_boto3.describe.training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name)['ModelArtifacts']['S3ModelArtifacts']

print("Model artifact persisted at " + artifact)


## Deployment

In [ ]:
artifact = "xxxxxxx"

from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name =  model_name,
    model_data=artifact,
    role = "XXXXXXXXXXXXX",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

In [ ]:
model_name

In [ ]:
## Endpoints deployment

endpoints_name = "Customer-sklearn-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Endpointname{endpoints_name}")

predictor = model.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.m4.xlarge',
    endpoint_name = endpoints_name
)

In [ ]:
endpoints_name

In [ ]:
predictor.endpoint_name

In [ ]:
predictor.predict(testX[features][1:2].values.tolist())

In [ ]:
result = predictor.predict(testX[features][:].values.tolist())

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, result)